[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Z-interpolation/IsoFlow/blob/main/manual/manual.ipynb)

# FlowDenoising User Manual

FlowDenoising is a low-pass filter of volumetric data. It uses a Gaussian kernel and the slices are warped using the optical flow, resulting shaper high-frequency structures. FlowDenoising comes with only one Python module (`flowdenoising.py`).

## Install requirements
Notice that these shell interactions are performed from the `manual` directory. From the `FlowDenoising`'s root directory, remove "`../`" when necessary.

In [11]:
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 40 kB 25 kB/s             


## Download a stack
This example uses the [EMPIAR-10311](https://www.ebi.ac.uk/empiar/EMPIAR-10310/) volume.

In [10]:
FILEID = "1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_"
FILENAME = "stack.mrc"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILEID}" -O {FILENAME} && rm -rf /tmp/cookies.txt
#!mv /tmp/stack.mrc .

--2022-06-11 13:30:45--  https://docs.google.com/uc?export=download&confirm=t&id=1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_
Cargado certificado CA '/etc/ssl/certs/ca-certificates.crt'
Resolviendo docs.google.com (docs.google.com)... 216.58.215.174, 2a00:1450:4003:806::200e
Conectando con docs.google.com (docs.google.com)[216.58.215.174]:443... conectado.
Petición HTTP enviada, esperando respuesta... 303 See Other
Localización: https://doc-08-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ic255cp5cs7n2uukbi3bc6jo4ltvomjn/1654947000000/01084993768099894447/*/1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_?e=download [siguiendo]
Aviso: no se admiten comodines en HTTP.
--2022-06-11 13:30:45--  https://doc-08-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ic255cp5cs7n2uukbi3bc6jo4ltvomjn/1654947000000/01084993768099894447/*/1lSFb4VZ5iqR3kLdPv1dDbdOe9dKPZ--_?e=download
Resolviendo doc-08-8s-docs.googleusercontent.com (doc-08-8s-docs.googleusercontent.com)

## Filtering a MRC file
Using default parameters

In [4]:
!python ../flowdenoising --input stack.mrc --output filtered_stack.mrc

(INFO) isoflow: renaming "./img_001.tif" to "./img_002.tif"
(INFO) isoflow: renaming "./img_000.tif" to "./img_000.tif"
(INFO) isoflow: reading image "./img_000.tif"
(INFO) isoflow: reading image "./img_002.tif"
(INFO) isoflow: CPU time = 1.128408541 seconds
(INFO) isoflow: writting image "./img_001.tif"


## Getting help

In [5]:
!python isoflow.py -h

usage: isoflow.py [-h] [-i IMAGES] [-f INTERPOLATION_FACTOR]
                  [-n NUMBER_OF_IMAGES]

Linear interpolation using Optical Flow.

options:
  -h, --help            show this help message and exit
  -i IMAGES, --images IMAGES
                        Input images filename template (default:
                        ./img_%03d.tif)
  -f INTERPOLATION_FACTOR, --interpolation_factor INTERPOLATION_FACTOR
                        Interpolation factor (default: 2)
  -n NUMBER_OF_IMAGES, --number_of_images NUMBER_OF_IMAGES
                        Number of input images (default: 2)


## Results

In [6]:
# "animate" requires imagemagick (https://imagemagick.org)
!animate img_00?.tif
# Type "q" for quiting animate / use "space" for pausing

In [7]:
# "convert" requires imagemagick (https://imagemagick.org)
!convert img_000.tif img_000.png
!convert img_001.tif img_001.png
!convert img_002.tif img_002.png

### img_000.tif (original image)
![img_000.png](./img_000.png "img_000.png")

### img_001.tif (interpolation image)
![img_001.png](./img_001.png "img_001.png")

### img_002.tif (original image)
![img_002.png](./img_002.png "img_002.png")